In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as iter_t

retail_data = [i.strip().split() for i in open("retail.dat").readlines()]

retail_data = pd.DataFrame(retail_data)

# allocate a limited number of rows, for testing
retail_data = retail_data[:100]

# Random Sampling

In [76]:
def random_sampling(data, min_support=0.05, min_confidence=0.10):
    
    total_transactions = len(data)
    
    reduction = 5
    
    data = data.sample(n=int(total_transactions/reduction))
    data.reset_index(inplace=True, drop=True)
    min_support = min_support / reduction
    
    def n_candidate(prev_L, n):
        names_helper = np.array([], dtype=int)
        for i in range(n-1):
            names_helper = np.append(names_helper, prev_L[i].to_numpy(dtype=int))
            
        candidate         = iter_t.combinations(set(names_helper), n)
        candidate         = pd.DataFrame([i for i in candidate])
        candidate["freq"] = np.zeros(len(candidate[0]), dtype=int)
        
        candidate.is_copy = False
        
        for i in range(data.shape[0]):
            for j in range(candidate.shape[0]):
                if set(candidate.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                    candidate.loc[j, "freq"] += 1
        return candidate
    
    # get unique items
    # construct array of all items
    items     = []
    hash_dict = {}
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            items.append(data[i][j])
        
    unique_items = list(set(items))
    
    # construct candidate sets
    C = []
    L = []
    
    C1 = []
    for val in unique_items:
        C1.append((val, items.count(val)))
    
    total_transactions = len(data)
    
    del(items)
    
    C1 = pd.DataFrame(C1, columns=[0, "freq"], dtype=int)
    C1 = C1.dropna()
    
    C1["conf"] = np.ones(len(C1[0]), dtype=float)
    C1["sup"]  = C1["freq"] / total_transactions
    
    L1 = C1[C1["sup"] >= min_support]
    L1 = L1.astype({0: int})
    
    C.append(C1)
    L.append(L1)
    
    def conf(_L, prevL, n):
        # Build confidence
        pd.set_option('mode.chained_assignment', 'warn')
        _L["conf"] = np.zeros(len(_L[0]), dtype=float)

        _L.is_copy = False
        for i in range(len(_L["freq"])):
            oldSup = _L["freq"].iloc[i]                                        #(prevL["freq"][prevL.iloc[:, 0:n] == L_cur[0:n]]).iat[0]
            for j in range(len(prevL[0])):
                if set(prevL.iloc[j, 0:n-1]) == set(_L.iloc[i, 0:n-1]):
                    oldSup = prevL["freq"].iloc[j]
                    break
            _L["conf"].iloc[i] = (_L["freq"].iloc[i] / oldSup)
        
        return _L
        
    
    i = 2
    while True:
        cand         = n_candidate(L[i-2], i)
        cand["sup"]  = cand["freq"] / total_transactions
        L_cur        = cand[cand["sup"] >= min_support]
        
        if len(L_cur[0]) == 0:
            break
        
        L_cur = conf(L_cur, L[i-2], i)
        
        L_cur = L_cur[L_cur["conf"] >= min_confidence]
        
        C.append(cand)
        L.append(L_cur)
        i += 1
    
    return L

In [78]:
res = random_sampling(retail_data, min_support=0.50) # threshold must be recalculated wrong? it makes it way too low
res

<ipython-input-76-ca454f328ba8>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _L["conf"] = np.zeros(len(_L[0]), dtype=float)
/home/howling/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/howling/.local/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

[       0  freq  conf   sup
 6    186     2   1.0  0.10
 8     18     2   1.0  0.10
 17    23     2   1.0  0.10
 21   258     2   1.0  0.10
 35    65     2   1.0  0.10
 42    48    12   1.0  0.60
 54   229     2   1.0  0.10
 69    15     2   1.0  0.10
 71   147     2   1.0  0.10
 79    39    12   1.0  0.60
 88    37     2   1.0  0.10
 91   101     2   1.0  0.10
 92   340     2   1.0  0.10
 105   66     2   1.0  0.10
 111   36     3   1.0  0.15
 118   41     4   1.0  0.20
 131   38     7   1.0  0.35
 138  347     2   1.0  0.10
 142   89     3   1.0  0.15,
        0    1  freq   sup      conf
 9    258   48     2  0.10  1.000000
 20    15   23     2  0.10  1.000000
 53   147   37     2  0.10  1.000000
 54   147   38     2  0.10  1.000000
 57   147   48     2  0.10  1.000000
 81    36   38     3  0.15  1.000000
 82    36   39     2  0.10  0.666667
 84    36   48     3  0.15  1.000000
 93    37   38     2  0.10  1.000000
 96    37   48     2  0.10  1.000000
 105   38   39     3  0.15  0.42

# SON